<a href="https://colab.research.google.com/github/Sneha-dasgupta/Multi-class-seg/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [57]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from tqdm import tqdm
!pip install -U albumentations
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    """ X = Images and Y = masks """

    train_x = sorted(glob(os.path.join(path, "Training", "Images", "*.png")))
    train_y1 = sorted(glob(os.path.join(path, "Training", "GT_OD", "*.png")))
    train_y2 = sorted(glob(os.path.join(path, "Training", "GT_OC", "*.png")))


    test_x = sorted(glob(os.path.join(path, "Test", "Images", "*.png")))
    test_y1 = sorted(glob(os.path.join(path, "Test", "Test_GT_OD", "*.png")))
    test_y2 = sorted(glob(os.path.join(path, "Test", "Test_GT_OC", "*.png")))

    return (train_x, train_y1, train_y2), (test_x, test_y1, test_y2)

def augment_data(images, mask1, mask2, save_path, augment=True):
    H = 256
    W = 256

    for idx, (x, y, z) in tqdm(enumerate(zip(images, mask1, mask2)), total=len(images)):
        """ Extracting names """
        name = x.split("/")[-1].split(".")[0]

        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        y = cv2.imread(y, cv2.IMREAD_COLOR)
        z = cv2.imread(z, cv2.IMREAD_COLOR)
        print(x.shape, y.shape, z.shape)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask1=y, mask2=z)
            x1 = augmented["image"]
            y1 = augmented["mask1"]
            z1 = augmented["mask2"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask1=y, mask2=z)
            x2 = augmented["image"]
            y2 = augmented["mask1"]
            z2 = augmented["mask2"]

            X = [x, x1, x2]
            Y = [y, y1, y2]
            Z = [z, z1, z2]

        else:
            X = [x]
            Y = [y]
            Z = [z]

        index = 0
        for i, m, n in zip(X, Y, Z):
            i = cv2.resize(i, (W, H), interpolation = cv2.INTER_CUBIC)
            #i = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)

            #Histogram equalisation - start
            #img_gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
            #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            #i = clahe.apply(img_gray)
            #i = cv2.cvtColor(i,cv2.COLOR_GRAY2RGB)
            lab = cv2.cvtColor(i, cv2.COLOR_BGR2LAB)

            lab_planes = cv2.split(lab)

            clahe = cv2.createCLAHE(clipLimit=3.0,tileGridSize=(8,8))

            lab_planes[0] = clahe.apply(lab_planes[0])

            lab = cv2.merge(lab_planes)

            i = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
            i = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)

            #histogram equalization - end 

            m = cv2.resize(m, (W, H), interpolation = cv2.INTER_CUBIC)
            n = cv2.resize(n, (W, H), interpolation = cv2.INTER_CUBIC)

            if len(X) == 1:
                tmp_image_name = f"{name}.png"
                tmp_mask1_name = f"{name}.png"
                tmp_mask2_name = f"{name}.png"
            else:
                tmp_image_name = f"{name}_{index}.png"
                tmp_mask1_name = f"{name}_{index}.png"
                tmp_mask2_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask1_path = os.path.join(save_path, "mask1", tmp_mask1_name)
            mask2_path = os.path.join(save_path, "mask2", tmp_mask2_name)

            #print(i.shape)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask1_path, m)
            cv2.imwrite(mask2_path, n)

            index += 1

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)

    """ Load the data """
    data_path = "/content/drive/MyDrive/Drishti/"
    (train_x, train_y1, train_y2), (test_x, test_y1, test_y2) = load_data(data_path)

    '''print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")'''

    """ Creating directories """
    create_dir("new_data/train/image")
    create_dir("new_data/train/mask1")
    create_dir("new_data/train/mask2")
    create_dir("new_data/test/image")
    create_dir("new_data/test/mask1")
    create_dir("new_data/test/mask2")

    augment_data(train_x, train_y1, train_y2, "new_data/train/", augment=False)
    augment_data(test_x, test_y1, test_y2, "new_data/test/", augment=False)

  3%|▎         | 2/79 [00:00<00:12,  6.28it/s]

(1752, 2045, 3) (1752, 2045, 3) (1752, 2045, 3)
(1762, 2049, 3) (1762, 2049, 3) (1762, 2049, 3)


  5%|▌         | 4/79 [00:00<00:12,  6.02it/s]

(1755, 2049, 3) (1755, 2049, 3) (1755, 2049, 3)
(1757, 2140, 3) (1757, 2140, 3) (1757, 2140, 3)


  8%|▊         | 6/79 [00:00<00:11,  6.20it/s]

...

(1750, 2048, 3) (1750, 2048, 3) (1750, 2048, 3)


 95%|█████████▌| 20/21 [00:03<00:00,  6.16it/s]

(1749, 2049, 3) (1749, 2049, 3) (1749, 2049, 3)


100%|██████████| 21/21 [00:03<00:00,  6.09it/s]

(1749, 2048, 3) (1749, 2048, 3) (1749, 2048, 3)


# CDR calculation

In [58]:
# FUNCTION TO CALCULATE CDR
import cv2 as cv

def cdr(cup,disc,plot):
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
    
    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)	
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img = clahe.apply(r3)
    
    
    ret,thresh = cv2.threshold(cup,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cup_diameter = 0
    largest_area = 0
    cup_area = 0
    el_cup = contours[0]
    if len(contours) != 0:
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv.contourArea(contours[i])

                if (area>largest_area):
                    largest_area=area
                    cup_area = largest_area
                    print("cup_area:", cup_area)
                    index = i
                    el_cup = cv.fitEllipse(contours[i])
                
    cv.ellipse(img,el_cup,(140,60,150),3)
    x,y,w,h = cv2.boundingRect(contours[index])
    cup_diameter = max(w,h)
    print("cup_diameter", cup_diameter)
    minor_axis = min(w,h)
    notch_factor = (cup_diameter - minor_axis)
    #print(notch_factor)
    
    
    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img2 = clahe.apply(r3)
    
    ret,thresh = cv.threshold(disc,127,255,0)
    contours,hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    disk_diameter = 0
    largest_area = 0
    disk_area = 0
    el_disc = el_cup
    if len(contours) != 0:
          for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv.contourArea(contours[i])
                if (area>largest_area):
                    largest_area=area
                    disk_area = largest_area
                    print("disk_area", disk_area)
                    index = i
                    el_disc = cv.fitEllipse(contours[i])
                    
    cv.ellipse(img2,el_disc,(140,60,150),3)
    x,y,w,h = cv2.boundingRect(contours[index])
    disk_diameter = max(w,h)
    print("disk_diameter", disk_diameter)
                
    if plot:
        plt.imshow(img2, cmap = 'gray', interpolation = 'bicubic')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.show()
        plt.imshow(img)
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()
        
    if(disk_diameter == 0): return 1
    dcdr = cup_diameter/disk_diameter
    acdr = cup_area/disk_area

    return dcdr, acdr, cup_diameter, disk_diameter, cup_area, disk_area

In [59]:
# FUNCTION TO CALCULATE disc diameter
import cv2 as cv

def disk_diameter(disc,plot):
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
   
    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img2 = clahe.apply(r3)
    
    ret,thresh = cv.threshold(disc,127,255,0)
    contours,hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    disk_diameter = 0
    largest_area = 0
    disk_area = 0
    el_disc = contours[0]
    if len(contours) != 0:
          for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv.contourArea(contours[i])
                if (area>largest_area):
                    largest_area=area
                    disk_area = largest_area
                    print("disk_area", disk_area)
                    index = i
                    el_disc = cv.fitEllipse(contours[i])
                    
    cv.ellipse(img2,el_disc,(140,60,150),3)
    x,y,w,h = cv2.boundingRect(contours[index])
    disk_diameter = max(w,h)
    print("disk_diameter", disk_diameter)
                
    if plot:
        plt.imshow(img2, cmap = 'gray', interpolation = 'bicubic')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.show()
        plt.imshow(img)
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()

    return disk_diameter

In [60]:
# FUNCTION TO CALCULATE NOTCH
import cv2 as cv
from skimage import transform 
from skimage import img_as_ubyte
import math
from scipy.spatial import distance
import statistics



def notch(cup,disc,plot,dd):
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
    
    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)	
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img = clahe.apply(r3)
    
    
    ret,thresh = cv2.threshold(cup,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

    thresh_c = thresh

    el_cup = contours[0]
    # calculate moments of binary image
    M = cv2.moments(el_cup)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    print(cX, cY)

    # Step #3
    out = cup.copy()

    # Step #4
    ref_c = np.zeros_like(thresh_c)
    cv2.drawContours(ref_c, contours, 0, 255, 1)

    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img2 = clahe.apply(r3)
    
    ret,thresh = cv.threshold(disc,127,255,0)
    contours,hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

    thresh_d = thresh

    el_disc = contours[0]
    # calculate moments of binary image
    N = cv2.moments(el_disc)
    # calculate x,y coordinate of center
    print(int(N["m00"]))
    if N["m00"] != 0:
        dX = int(N["m10"] / N["m00"])
        dY = int(N["m01"] / N["m00"])
    else:
    # set values as what you need in the situation
        dX, dY = 0, 0
    print(dX, dY)

    # Step #3
    out_d = disc.copy()

    # Step #4
    ref_d = np.zeros_like(thresh_d)
    cv2.drawContours(ref_d, contours, 0, 255, 1)

    # Get dimensions of the image
    width_c = cup.shape[1]
    height_c = cup.shape[0]
    width_d = disc.shape[1]
    height_d = disc.shape[0]

    Rtimax = 0
    Rti = []
    Rtsmax = 0
    Rts = []
    Rttmax = 0
    Rtt = []
    Rtnmax = 0
    Rtn = []

    for i in range(0,361):
        tmp_c = np.zeros_like(thresh_c)
        tmp_d = np.zeros_like(thresh_d)

        # Step #6b
        theta = (i)
        theta *= np.pi/180.0

        # Step #6c
        cv2.line(tmp_c, (cX, cY),
                (int(cX+np.cos(theta)*width_c),
                int(cY-np.sin(theta)*height_c)), 255, 5)
        if (dX==0 and dY==0):
            print("")
            #col_d[0],row_d[0] = 0, 0
        else:
            cv2.line(tmp_d, (dX, dY),
                    (int(dX+np.cos(theta)*width_d),
                    int(dY-np.sin(theta)*height_d)), 255, 5)


        # Step #6d
        (row_c,col_c) = np.nonzero(np.logical_and(tmp_c, ref_c))
        (row_d,col_d) = np.nonzero(np.logical_and(tmp_d, ref_d))

        # Step #6e
        #cv2.line(out, (cX, cY), (col_c[0],row_c[0]), 0, 1)
        #cv2.line(out_d, (dX, dY), (col_d[0],row_d[0]), 0, 1)


        #print("Boundary co-ordinate for cup", col_c[0],row_c[0])
        #print("Boundary co-ordinate for disc", col_d[0],row_d[0])
        if ((dX != 0) and (dY !=0)):
            Dc = distance.euclidean([cX, cY], [col_c[0],row_c[0]])
            Dd = distance.euclidean([dX, dY], [col_d[0],row_d[0]])
            #print("Dc", Dc)
            #print("Dd", Dd)
            Rt = Dd - Dc
            Rt = Rt/dd
            #print("Rt", Rt)
            if ((0< i <46) or (315< i <361)):
                Rtimax = Rtimax + Rt
                Rti.append(Rt)
             
            elif (135< i <226):
                Rtsmax = Rtsmax + Rt
                Rts.append(Rt)
             
            elif (45< i <136):
                Rttmax = Rttmax + Rt
                Rtt.append(Rt)
           
            else:
                Rtnmax = Rtnmax + Rt
                Rtn.append(Rt)
        else:
            #print("discard") 
            Rt = 0
            Rti.append(Rt)   
            Rts.append(Rt)
            Rtt.append(Rt)
            Rtn.append(Rt)


    print("Rtimax =", Rtimax)
    print("Rtsmax =", Rtsmax)
    print("Rttmax =", Rttmax)
    print("Rtnmax =", Rtnmax)
    i = statistics.mean(Rti)
    print("mean(i)", i)
    s = statistics.mean(Rts)
    print("mean(s)", s)
    n = statistics.mean(Rtn)
    print("mean(n)", n)
    t = statistics.mean(Rtt)
    print("mean(t)", t)


    return i, s

# Train Dataset Features Extraction

In [61]:
# MAIN FUNCTION

import pandas as pd

CDR = []
VAL = []
count = 0

folder_train_y1 = "/content/new_data/train/mask1/"
train_y1 = os.listdir(folder_train_y1)
train_y1.sort(key = len)

folder_train_y2 = "/content/new_data/train/mask2/"
train_y2 = os.listdir(folder_train_y2)
train_y2.sort(key = len)

folder_train_y3 = "/content/new_data/train/image/"
train_y3 = os.listdir(folder_train_y3)
train_y3.sort(key = len)

create_dir("results")
SCORE = []

for x, y, z in tqdm(zip(train_y1, train_y2, train_y3 ), total=len(train_y1)):
    """ Extracting names """
    name = x.split("/")[-1].split(".")[0]
    disc = cv2.imread(folder_train_y1+x,0)

    cup = cv2.imread(folder_train_y2+y,0)

    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    dcdr_cal, acdr_cal, cup_d, disc_d, cup_a, disc_a = cdr(cup,disc,False)
    dd = disk_diameter(disc,False)
    i, s = notch(cup,disc,False,dd)
    print("DCDR", dcdr_cal)
    print("ACDR", acdr_cal)
    print(name)
    print("I-distance", i)
    print("S-distance", s)

    Name = name
    ACDR = acdr_cal
    DCDR = dcdr_cal
    cup_diameter = cup_d
    disc_diameter = disc_d
    cup_area = cup_a
    disc_area = disc_a
    i_distance = i
    s_distance = s

    SCORE.append([Name, ACDR, DCDR, cup_diameter, disc_diameter, cup_area, disc_area, i_distance, s_distance])

    count = count + 1

    SCORE.sort(reverse=False)
    df = pd.DataFrame(SCORE, columns=["Name", "ACDR", "DCDR", "Cup diameter", "Disc diameter", "Cup area", "Disc area", "I distance", "S distance"])
    df.to_csv("results/extracted_features_train.csv")

  0%|          | 0/79 [00:00<?, ?it/s]

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 1673.5
cup_diameter 53
disk_area 2159.5
disk_diameter 61
disk_area 2159.5
disk_diameter 61
109 132
2159
110 132


  1%|▏         | 1/79 [00:00<00:19,  3.92it/s]

Rtimax = 4.746443732571681
Rtsmax = 4.209872147119586
Rttmax = 6.079841349567143
Rtnmax = 4.0980944306260305
mean(i) 0.052738263695240886
mean(s) 0.04677635719021761
mean(n) 0.045034004732154194
mean(t) 0.06755379277296829
DCDR 0.8688524590163934
ACDR 0.774947904607548
drishtiGS_076
I-distance 0.052738263695240886
S-distance 0.04677635719021761
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 1240.5
cup_diameter 45
disk_area 2347.5
disk_diameter 62
disk_area 2347.5
disk_diameter 62
120 143
2347
121 144


  3%|▎         | 2/79 [00:00<00:19,  4.00it/s]

...

Rtimax = 7.148590485814756
Rtsmax = 7.721088324207081
Rttmax = 8.377098694113863
Rtnmax = 7.9144763079688625
mean(i) 0.07942878317571953
mean(s) 0.08578987026896757
mean(n) 0.08697226712053696
mean(t) 0.09307887437904294
DCDR 0.8269230769230769
ACDR 0.6592724046140195
drishtiGS_004
I-distance 0.07942878317571953
S-distance 0.08578987026896757
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 963.5
cup_diameter 41
disk_area 1527.0
disk_diameter 52
disk_area 1527.0
disk_diameter 52
141 117
1527
141 116


 99%|█████████▊| 78/79 [00:19<00:00,  3.92it/s]

Rtimax = 7.294923462258379
Rtsmax = 7.024094694150628
Rttmax = 7.638993114828815
Rtnmax = 9.336522168228363
mean(i) 0.08105470513620422
mean(s) 0.07804549660167366
mean(n) 0.10259914470580618
mean(t) 0.08487770127587572
DCDR 0.7884615384615384
ACDR 0.6309757694826457
drishtiGS_051
I-distance 0.08105470513620422
S-distance 0.07804549660167366
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 410.0
cup_diameter 27
disk_area 1584.5
disk_diameter 51
disk_area 1584.5
disk_diameter 51
143 162
1584
143 161


100%|██████████| 79/79 [00:19<00:00,  3.97it/s]

Rtimax = 18.48271945750266
Rtsmax = 18.885414423584265
Rttmax = 18.640947149346363
Rtnmax = 21.804760940484925
mean(i) 0.20536354952780736
mean(s) 0.20983793803982517
mean(n) 0.23961275758774653
mean(t) 0.20712163499273736
DCDR 0.5294117647058824
ACDR 0.2587567055853582
drishtiGS_047
I-distance 0.20536354952780736
S-distance 0.20983793803982517


# Test Dataset Features Extraction

In [62]:
# MAIN FUNCTION

import pandas as pd

CDR = []
VAL = []
count = 0

folder_test_y1 = "/content/new_data/test/mask1/"
test_y1 = os.listdir(folder_test_y1)
test_y1.sort(key = len)

folder_test_y2 = "/content/new_data/test/mask2/"
test_y2 = os.listdir(folder_test_y2)
test_y2.sort(key = len)

folder_test_y3 = "/content/new_data/test/image/"
test_y3 = os.listdir(folder_test_y3)
test_y3.sort(key = len)

#create_dir("results")
SCORE_TEST = []

for x, y, z in tqdm(zip(test_y1, test_y2, test_y3 ), total=len(test_y1)):
    """ Extracting names """
    name = x.split("/")[-1].split(".")[0]
    disc = cv2.imread(folder_test_y1+x,0)
    cup = cv2.imread(folder_test_y2+y,0)
    #print(disc.shape)
    #print(cup.shape)
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    dcdr_cal, acdr_cal, cup_d, disc_d, cup_a, disc_a = cdr(cup,disc,False)
    dd = disk_diameter(disc,False)
    i, s = notch(cup,disc,False,dd)

    print("DCDR", dcdr_cal)
    print("ACDR", acdr_cal)
    print(name)
    print("I-distance", i)
    print("S-distance", s)

    Name = name
    ACDR = acdr_cal
    DCDR = dcdr_cal
    cup_diameter = cup_d
    disc_diameter = disc_d
    cup_area = cup_a
    disc_area = disc_a
    i_distance = i
    s_distance = s
    SCORE_TEST.append([Name, ACDR, DCDR, cup_diameter, disc_diameter, cup_area, disc_area, i_distance, s_distance])

    count = count + 1

    SCORE_TEST.sort(reverse=False)
    df = pd.DataFrame(SCORE_TEST, columns=["Name", "ACDR", "DCDR", "Cup diameter", "Disc diameter", "Cup area", "Disc area", "I distance", "S distance"])
    df.to_csv("results/extracted_features_test.csv")

  0%|          | 0/21 [00:00<?, ?it/s]

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 2355.0
cup_diameter 64
disk_area 3441.0
disk_diameter 76
disk_area 3441.0
disk_diameter 76
111 148
3441
112 147


  5%|▍         | 1/21 [00:00<00:05,  3.98it/s]

Rtimax = 6.364789082955187
Rtsmax = 6.222616933372714
Rttmax = 7.34352755300426
Rtnmax = 7.535736558926675
mean(i) 0.07071987869950208
mean(s) 0.06914018814858572
mean(n) 0.08281029185633708
mean(t) 0.08159475058893624
DCDR 0.8421052631578947
ACDR 0.6843940714908456
drishtiGS_005
I-distance 0.07071987869950208
S-distance 0.06914018814858572
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 1655.5
cup_diameter 52
disk_area 2668.5
disk_diameter 65
disk_area 2668.5
disk_diameter 65
130 128
2668
129 128


 10%|▉         | 2/21 [00:00<00:04,  4.03it/s]

...

Rtimax = 4.37813178992875
Rtsmax = 4.6581293327221545
Rttmax = 6.5430586546694345
Rtnmax = 6.25910268000362
mean(i) 0.04864590877698611
mean(s) 0.05175699258580172
mean(n) 0.06878134813190798
mean(t) 0.07270065171854925
DCDR 0.8771929824561403
ACDR 0.7235363690124187
drishtiGS_025
I-distance 0.04864590877698611
S-distance 0.05175699258580172
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 904.5
cup_diameter 38
disk_area 2402.0
disk_diameter 62
disk_area 2402.0
disk_diameter 62
102 143
2402
103 142


 95%|█████████▌| 20/21 [00:05<00:00,  4.01it/s]

Rtimax = 14.713034385603455
Rtsmax = 13.96117007692151
Rttmax = 15.316929388457527
Rtnmax = 17.159148412445525
mean(i) 0.16347815984003833
mean(s) 0.15512411196579454
mean(n) 0.18856207046643436
mean(t) 0.17018810431619472
DCDR 0.6129032258064516
ACDR 0.3765611990008326
drishtiGS_098
I-distance 0.16347815984003833
S-distance 0.15512411196579454
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 1076.5
cup_diameter 43
disk_area 1579.0
disk_diameter 52
disk_area 1579.0
disk_diameter 52
115 134
1579
116 134


100%|██████████| 21/21 [00:05<00:00,  3.97it/s]

Rtimax = 5.518815804512246
Rtsmax = 6.09241201333485
Rttmax = 6.893823124412758
Rtnmax = 8.18624262654887
mean(i) 0.06132017560569163
mean(s) 0.06769346681483167
mean(n) 0.08995871018185572
mean(t) 0.07659803471569736
DCDR 0.8269230769230769
ACDR 0.681760607979734
drishtiGS_011
I-distance 0.06132017560569163
S-distance 0.06769346681483167


# Predicted Dataset Features Extraction

In [63]:
# MAIN FUNCTION

import pandas as pd

CDR = []
VAL = []
count = 0

folder_pred_y1 = "/content/drive/MyDrive/Project(Final)/Segmentation/Cup/Results_Cup/results/Segmented Cup/"
pred_y1 = os.listdir(folder_pred_y1)
pred_y1.sort(key = len)

folder_pred_y2 = "/content/drive/MyDrive/Project(Final)/Segmentation/Disc/results_disc/content/results/segmented disc/"
pred_y2 = os.listdir(folder_pred_y2)
pred_y2.sort(key = len)


#create_dir("results")
SCORE_PRED = []

for x, y in tqdm(zip(pred_y1, pred_y2), total=len(pred_y1)):
    """ Extracting names """
    name = x.split("/")[-1].split(".")[0]
    disc = cv2.imread(folder_pred_y2+y,0)
    disc = cv2.resize(disc,(256,256), interpolation = cv2.INTER_NEAREST)
    #disc = cv2.cvtColor(disc, cv2.COLOR_RGB2GRAY)
    cup = cv2.imread(folder_pred_y1+x,0)
    cup = cv2.resize(cup,(256,256), interpolation = cv2.INTER_NEAREST)
    print(cup.shape)
    #cup = cv2.resize(cup,(256,256), interpolation = cv2.INTER_NEAREST)
    #cup = cv2.cvtColor(cup, cv2.COLOR_RGB2GRAY)
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    dcdr_cal, acdr_cal, cup_d, disc_d, cup_a, disc_a = cdr(cup,disc,False)
    dd = disk_diameter(disc,False)
    i, s = notch(cup,disc,False,dd)
    print("DCDR", dcdr_cal)
    print("ACDR", acdr_cal)
    print(name)
    print("I-distance", i)
    print("S-distance", s)

    Name = name
    ACDR = acdr_cal
    DCDR = dcdr_cal
    cup_diameter = cup_d
    disc_diameter = disc_d
    cup_area = cup_a
    disc_area = disc_a
    i_distance = i
    s_distance = s
    SCORE_PRED.append([Name, ACDR, DCDR, cup_diameter, disc_diameter, cup_area, disc_area, i_distance, s_distance])

    count = count + 1

    SCORE_PRED.sort(reverse=False)
    df = pd.DataFrame(SCORE_PRED, columns=["Name", "ACDR", "DCDR", "Cup diameter", "Disc diameter", "Cup area", "Disc area", "I distance", "S distance"])
    df.to_csv("results/extracted_features_pred.csv")

  0%|          | 0/20 [00:00<?, ?it/s]

(256, 256)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 380.0
cup_diameter 26
disk_area 1533.5
disk_diameter 50
disk_area 1533.5
disk_diameter 50
120 148
1533
119 147


  5%|▌         | 1/20 [00:00<00:05,  3.67it/s]

Rtimax = 16.856046563388507
Rtsmax = 18.76393844602118
Rttmax = 21.93834761246273
Rtnmax = 22.38661415509089
mean(i) 0.18728940625987234
mean(s) 0.2084882049557909
mean(n) 0.24600674895704266
mean(t) 0.24375941791625272
DCDR 0.52
ACDR 0.24779915226605803
drishtiGS_094
I-distance 0.18728940625987234
S-distance 0.2084882049557909
(256, 256)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 919.0
cup_diameter 38
disk_area 2427.0
disk_diameter 62
disk_area 2427.0
disk_diameter 62
103 143
2427
104 143


 10%|█         | 2/20 [00:00<00:04,  3.60it/s]

...

Rtimax = 5.478122299495033
Rtsmax = 4.337019549986079
Rttmax = 7.519474084686375
Rtnmax = 7.884113031274604
mean(i) 0.060868025549944846
mean(s) 0.048189106110956455
mean(n) 0.08663860473928135
mean(t) 0.08354971205207086
DCDR 0.8305084745762712
ACDR 0.6940673705379587
drishtiGS_003
I-distance 0.060868025549944846
S-distance 0.048189106110956455
(256, 256)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 1772.0
cup_diameter 54
disk_area 1529.5
disk_diameter 51
disk_area 1529.5
disk_diameter 51
125 132
1529
124 141


 95%|█████████▌| 19/20 [00:05<00:00,  3.82it/s]

Rtimax = -1.09476804541778
Rtsmax = -4.661214971039846
Rttmax = -2.7875054215908617
Rtnmax = -3.0558990107546835
mean(i) -0.012164089393530897
mean(s) -0.051791277455998305
mean(n) -0.03358130781049105
mean(t) -0.030972282462120717
DCDR 1.0588235294117647
ACDR 1.158548545276234
drishtiGS_020
I-distance -0.012164089393530897
S-distance -0.051791277455998305
(256, 256)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cup_area: 1073.5
cup_diameter 44
disk_area 2069.0
disk_diameter 60
disk_area 2069.0
disk_diameter 60
166 128
2069
123 128


100%|██████████| 20/20 [00:05<00:00,  3.64it/s]

Rtimax = 10.153131560183345
Rtsmax = 9.201738979832536
Rttmax = 11.73109039669587
Rtnmax = 11.693361924269835
mean(i) 0.11281257289092608
mean(s) 0.10224154422036148
mean(n) 0.12849848268428402
mean(t) 0.1303454488521765
DCDR 0.7333333333333333
ACDR 0.5188496858385694
drishtiGS_013
I-distance 0.11281257289092608
S-distance 0.10224154422036148


# X_train

In [64]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [65]:
# Importing the dataset
dataset = pd.read_csv('/content/results/extracted_features_train.csv')
X_train = dataset.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9]].values
print(X_train)


[[5.73057733e-01 8.08510638e-01 3.80000000e+01 4.70000000e+01
  8.04000000e+02 1.40300000e+03 1.07299110e-01 1.12310834e-01]
 [6.59272405e-01 8.26923077e-01 4.30000000e+01 5.20000000e+01
  1.11450000e+03 1.69050000e+03 7.94287832e-02 8.57898703e-02]
 [3.35081585e-01 5.96153846e-01 3.10000000e+01 5.20000000e+01
  5.75000000e+02 1.71600000e+03 1.67084763e-01 1.81544401e-01]
 [7.70623742e-01 8.87096774e-01 5.50000000e+01 6.20000000e+01
  1.72350000e+03 2.23650000e+03 4.37295165e-02 6.28842007e-02]
 [7.57688723e-01 8.67924528e-01 4.60000000e+01 5.30000000e+01
  1.21950000e+03 1.60950000e+03 6.12536602e-02 4.21481025e-02]
 [7.23741794e-01 9.24528302e-01 4.90000000e+01 5.30000000e+01
  1.32300000e+03 1.82800000e+03 7.23758966e-02 8.64532759e-02]
 [7.05164319e-01 9.03846154e-01 4.70000000e+01 5.20000000e+01
  1.12650000e+03 1.59750000e+03 9.25267778e-02 6.76601491e-02]
 [3.56485604e-01 5.84905660e-01 3.10000000e+01 5.30000000e+01
  6.00500000e+02 1.68450000e+03 1.46905143e-01 1.67988054e-01]


# Y_train (Labels)

In [66]:
from scipy import signal
import cv2
import sys
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import os
import xlrd 
import math
from pylab import*
from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage import data

In [67]:
wb = xlrd.open_workbook("/content/drive/MyDrive/Drishti/Drishti-GS1_diagnosis.xlsx") 
sheet = wb.sheet_by_index(0) 
row_count = sheet.nrows
col_count = sheet.ncols
print(row_count)
print(col_count)

80
9


In [68]:
CELL = []
Val=0
for cur_row in range(1, row_count):
        cell1 = sheet.cell(cur_row, 1)
        cell2 = sheet.cell(cur_row, 8)
        if (cell2.value == 'Glaucomatous'):
              Val = 1
        else:
              Val = 0
        print(cell1.value)
        print(cell2.value)
        print(Val)
        Name = cell1.value
        Total = cell2.value
        Class = Val
        CELL.append([Name, Total, Class])       

CELL.sort(reverse=False)
df = pd.DataFrame(CELL, columns=[ "Name", "Total", "Class"])
df.to_csv("results/Class_train.csv")  

drishtiGS_002
Glaucomatous
1
drishtiGS_004
Glaucomatous
1
drishtiGS_008
Normal
0
drishtiGS_010
Glaucomatous
1
drishtiGS_012
Glaucomatous
1
drishtiGS_015
Glaucomatous
1
drishtiGS_016
Glaucomatous
1
drishtiGS_017
Normal
0
drishtiGS_018
Normal
0
drishtiGS_022
Glaucomatous
1
drishtiGS_024
Glaucomatous
1
drishtiGS_026
Glaucomatous
1
drishtiGS_031
Glaucomatous
1
drishtiGS_032
Glaucomatous
1
drishtiGS_033
Normal
0
drishtiGS_035
Normal
0
drishtiGS_036
Normal
0
drishtiGS_037
Normal
0
drishtiGS_038
Glaucomatous
1
drishtiGS_040
Glaucomatous
1
drishtiGS_041
Normal
0
drishtiGS_042
Normal
0
drishtiGS_043
Glaucomatous
1
drishtiGS_044
Glaucomatous
1
drishtiGS_045
Glaucomatous
1
drishtiGS_046
Normal
0
drishtiGS_047
Normal
0
drishtiGS_048
Glaucomatous
1
drishtiGS_049
Glaucomatous
1
drishtiGS_050
Glaucomatous
1
drishtiGS_051
Glaucomatous
1
drishtiGS_052
Glaucomatous
1
drishtiGS_053
Glaucomatous
1
drishtiGS_054
Glaucomatous
1
drishtiGS_055
Glaucomatous
1
drishtiGS_056
Glaucomatous
1
drishtiGS_057
Normal
0

In [69]:
# Importing the dataset
dataset = pd.read_csv('/content/results/Class_train.csv')
Y_train = dataset.iloc[:, [3]].values
print(Y_train)

[[1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


# Y_test (Labels for GT)

In [70]:
wb = xlrd.open_workbook("/content/drive/MyDrive/Drishti/DrishtiGS_diagnosis_test.xlsx") 
sheet = wb.sheet_by_index(0) 
row_count = sheet.nrows
col_count = sheet.ncols
print(row_count)
print(col_count)

21
9


In [71]:
CELL = []
Val=0
for cur_row in range(1, row_count):
        cell1 = sheet.cell(cur_row, 1)
        cell2 = sheet.cell(cur_row, 8)
        if (cell2.value == 'Glaucomatous'):
              Val = 1
        else:
              Val = 0
        print(cell1.value)
        print(cell2.value)
        print(Val)
        Name = cell1.value
        Total = cell2.value
        Class = Val
        CELL.append([Name, Total, Class])       

CELL.sort(reverse=False)
df = pd.DataFrame(CELL, columns=[ "Name", "Total", "Class"])
df.to_csv("results/Class_test.csv")     

drishtiGS_001
Glaucomatous
1
drishtiGS_003
Glaucomatous
1
drishtiGS_006
Glaucomatous
1
drishtiGS_007
Normal
0
drishtiGS_009
Normal
0
drishtiGS_011
Glaucomatous
1
drishtiGS_013
Normal
0
drishtiGS_014
Glaucomatous
1
drishtiGS_019
Glaucomatous
1
drishtiGS_020
Glaucomatous
1
drishtiGS_021
Glaucomatous
1
drishtiGS_025
Glaucomatous
1
drishtiGS_028
Glaucomatous
1
drishtiGS_030
Glaucomatous
1
drishtiGS_034
Glaucomatous
1
drishtiGS_093
Normal
0
drishtiGS_094
Normal
0
drishtiGS_095
Normal
0
drishtiGS_096
Normal
0
drishtiGS_098
Normal
0


In [72]:
# Importing the dataset
dataset = pd.read_csv('/content/results/Class_test.csv')
Y_test = dataset.iloc[:, [3]].values
print(Y_test)
print(Y_test.shape)

[[1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
(20, 1)


# X_test (obtained from predicted images)

In [73]:
# Importing the dataset
dataset = pd.read_csv('/content/results/extracted_features_pred.csv')
X_test = dataset.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9]].values
print(X_test.shape)

(20, 8)


# SVM

In [74]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier_svm = SVC(kernel = 'rbf', random_state = 0, class_weight={0:4, 1:1})
classifier_svm.fit(X_train, np.ravel(Y_train, order='C'))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight={0: 4, 1: 1},
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

# Random Forest

In [75]:
# Fitting RF to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier_rd = RandomForestClassifier(n_estimators = 400, criterion = "gini")
classifier_rd.fit(X_train, np.ravel(Y_train, order='C'))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# KNN

In [76]:
# Fitting knn to the Training set
from sklearn.neighbors import KNeighborsClassifier  
classifier_knn = KNeighborsClassifier(n_neighbors=3, p=3)  
classifier_knn.fit(X_train, np.ravel(Y_train, order='C'))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=3,
                     weights='uniform')

# y_pred (Predicting labels)

In [77]:
# Predicting the Test set results
y_pred_svm = classifier_svm.predict(X_test)

In [78]:
print(np.ravel(Y_test, order='C'))

[1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0]


In [79]:
# Predicting the Test set results
y_pred_rd = classifier_rd.predict(X_test)

In [80]:
# Predicting the Test set results
y_pred_knn = classifier_knn.predict(X_test)

In [81]:
print(y_pred_svm)
print(y_pred_svm.shape)

[1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
(20,)


In [82]:
print(y_pred_rd)
print(y_pred_rd.shape)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0]
(20,)


In [83]:
print(y_pred_knn)
print(y_pred_knn.shape)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0]
(20,)


# Confusion Matrix

In [84]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred_svm)
print(cm)


[[ 7  1]
 [ 0 12]]


In [85]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_knn = confusion_matrix(Y_test, y_pred_knn)
print(cm_knn)

[[ 5  3]
 [ 0 12]]


In [86]:
tn, fp, fn, tp = cm.ravel()
sen=tp/(tp+fn)
print('\nSensitivity =', sen)
spec= tn/(tn+fp)
print('\nSpecificity =', spec)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print('\nAccuracy =', accuracy,'\n')


Sensitivity = 1.0

Specificity = 0.875

Accuracy = 0.95 



In [87]:
tn, fp, fn, tp = cm_knn.ravel()
sen=tp/(tp+fn)
print('\nSensitivity =', sen)
spec= tn/(tn+fp)
print('\nSpecificity =', spec)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print('\nAccuracy =', accuracy,'\n')


Sensitivity = 1.0

Specificity = 0.625

Accuracy = 0.85 



In [88]:
print("Model = Classification")
from sklearn.metrics import classification_report, confusion_matrix
print('\nClassification Report')
target_names = ['Normal', 'Glaucomatous']
print(classification_report(Y_test, y_pred_svm, target_names=target_names))



Model = Classification

Classification Report
              precision    recall  f1-score   support

      Normal       1.00      0.88      0.93         8
Glaucomatous       0.92      1.00      0.96        12

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



# Mischelleneous

In [ ]:
import pandas as pd

CDR = []
VAL = []
count = 0

folder_train_y1 = "/content/new_data/train/mask1/"
train_y1 = os.listdir(folder_train_y1)
train_y1.sort(key = len)

folder_train_y2 = "/content/new_data/train/mask2/"
train_y2 = os.listdir(folder_train_y2)
train_y2.sort(key = len)

folder_train_y3 = "/content/new_data/train/image/"
train_y3 = os.listdir(folder_train_y3)
train_y3.sort(key = len)

create_dir("results")
SCORE = []

for x, y, z in tqdm(zip(train_y1, train_y2, train_y3 ), total=len(train_y1)):
    """ Extracting names """
    name = x.split("/")[-1].split(".")[0]
    disc = cv2.imread(folder_train_y1+x,0)

    cup = cv2.imread(folder_train_y2+y,0)
    if (name != 'drishtiGS_097'):
        print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        dd = disk_diameter(disc,False)
        i, s = notch(cup,disc,False,dd)
        print("S distance", s)
        print("I distance", i)
        print(name)
    else:
      print("discard")


In [ ]:
    cv2.drawContours(img_as_ubyte(cup), el_cup, -1, (0, 255, 0), 2)
    cv2.circle(img_as_ubyte(cup), (cX, cY), 1, (0, 0, 255), 10)
    cv2.putText(img_as_ubyte(cup), "center", (cX - 20, cY - 20),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    plt.imshow(cup)
    plt.axis("off")
    plt.title("Optic Cup")
    plt.show()

    cv2.drawContours(img_as_ubyte(disc), el_disc, -1, (0, 255, 0), 2)
    cv2.circle(img_as_ubyte(disc), (dX, dY), 1, (0, 0, 255), 10)
    cv2.putText(img_as_ubyte(disc), "center", (dX - 20, dY - 20),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    plt.imshow(disc)
    plt.axis("off")
    plt.title("Optic Disc")
    plt.show()

In [ ]:
# FUNCTION TO CALCULATE NOTCH
import cv2 as cv
from skimage import transform 
from skimage import img_as_ubyte
from skimage.viewer import ImageViewer


def notch(cup,disc,plot):
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
    
    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)	
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img = clahe.apply(r3)
    
    
    ret,thresh = cv2.threshold(cup,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

    thresh_c = thresh

    el_cup = contours[0]
    # calculate moments of binary image
    M = cv2.moments(el_cup)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] /M["m00"])
    print(cX, cY)

    # Step #3
    out = cup.copy()

    # Step #4
    ref_c = np.zeros_like(thresh_c)
    cv2.drawContours(ref_c, contours, 0, 255, 1)

    # Get dimensions of the image
    width_c = cup.shape[1]
    height_c = cup.shape[0]

    tmp_c = np.zeros_like(thresh_c)

    # Step #6b
    theta = (360/6)
    theta *= np.pi/180.0

    # Step #6c
    cv2.line(tmp_c, (cX, cY),
            (int(cX+np.cos(theta)*width_c),
            int(cY-np.sin(theta)*height_c)), 255, 5)

    # Step #6d
    (row_c,col_c) = np.nonzero(np.logical_and(tmp_c, ref_c))

    # Step #6e
    cv2.line(out, (cX, cY), (col_c[0],row_c[0]), 0, 1)

    print("Boundary co-ordinate", col_c[0],row_c[0])
   

    plt.imshow(out)
    plt.axis("off")
    plt.title("Optic Cup")
    plt.show()

   
    
    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    #f4 = cv2.subtract(R3,img)
    img2 = clahe.apply(r3)
    
    ret,thresh = cv.threshold(disc,127,255,0)
    contours,hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

    el_disc = contours[0]
    # calculate moments of binary image
    N = cv2.moments(el_disc)
    # calculate x,y coordinate of center
    print(int(N["m00"]))
    if N["m00"] != 0:
        dX = int(N["m10"] / N["m00"])
        dY = int(N["m01"] / N["m00"])
    else:
    # set values as what you need in the situation
        dX, dY = 0, 0
    #dX = int(N["m10"] / N["m00"])
    #dY = int(N["m01"] / N["m00"])
    print(dX, dY)

    # Step #3
    out_d = disc.copy()

    # Step #4
    ref = np.zeros_like(thresh)
    cv2.drawContours(ref, contours, 0, 255, 1)

    # Get dimensions of the image
    width = disc.shape[1]
    height = disc.shape[0]

    tmp = np.zeros_like(thresh)

    # Step #6b
    theta = (360/6)
    theta *= np.pi/180.0

    # Step #6c
    if (dX==0 and dY==0):
      print("Discard this case")
      col[0],row[0] = 0, 0
    else:
      cv2.line(tmp, (dX, dY),
            (int(dX+np.cos(theta)*width),
            int(dY-np.sin(theta)*height)), 255, 5)

      # Step #6d
      (row,col) = np.nonzero(np.logical_and(tmp, ref))

      # Step #6e
      cv2.line(out_d, (dX, dY), (col[0],row[0]), 0, 1)

    print("Boundary co-ordinate", col[0],row[0])

    plt.imshow(out_d)
    plt.axis("off")
    plt.title("Optic Disk")
    plt.show()


    return cX, cY, dX, dY